In [35]:
import os
import random
import pickle
import datetime

import torch
import torchmetrics
import torchsummary
import numpy as np
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import utils

In [2]:
embeder = utils.PhotoEmbedingStorage('emb storage.pkl')

In [3]:
emb1 = embeder[13550]
emb2 = embeder[2169]
print('similarity', emb1 @ emb2.T)

similarity tensor([[0.7698]])


In [4]:
emb1 = embeder[13543]
emb2 = embeder[13544]
print('similarity', emb1 @ emb2.T)

similarity tensor([[0.6263]])


In [5]:
emb1 = embeder[13546]
emb2 = embeder[13547]
print('similarity', emb1 @ emb2.T)

similarity tensor([[0.5527]])


In [6]:
emb1 = embeder[13543]
emb2 = embeder[13550]
print('similarity', emb1 @ emb2.T)

similarity tensor([[0.3056]])


In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBED_DIM = 32

In [8]:
DEVICE = 'cpu'

In [9]:
files_opposite = []
files_target = []

path = "data/markup_opposite/"
files_opposite += [int(f.split('.')[0]) for f in os.listdir(path)]
    
path =  "data/markup_target/"
files_target += [int(f.split('.')[0]) for f in os.listdir(path)]

path = "data/opposite/"
files_opposite += [int(f.split('.')[0]) for f in os.listdir(path)]

path =  "data/target/"
files_target += [int(f.split('.')[0]) for f in os.listdir(path)]

files_opposite = np.array(files_opposite)
files_target = np.array(files_target)

y_opposite = np.zeros_like(files_opposite, dtype='float32')
y_target = np.ones_like(files_target, dtype='float32')

X = np.concatenate([files_opposite, files_target])
Y = np.concatenate([y_opposite, y_target])

Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.2, random_state=69)

In [39]:
Xtest = []
Ytest = []

path = "data/test_target/"
ldir = [int(f.split('.')[0]) for f in os.listdir(path)]
Xtest+= ldir
Ytest+= [1. for _ in ldir]
    
path = "data/test_opposite/"
ldir = [int(f.split('.')[0]) for f in os.listdir(path)]
Xtest+= ldir
Ytest+= [0. for _ in ldir]

Xtest = np.array(Xtest)
Ytest = np.array(Ytest, dtype='float32')

In [12]:
embeds = np.concatenate([embeder[int(xid)] for xid in Xtrain])

In [36]:
pca = PCA(n_components=EMBED_DIM)
pca.fit(embeds)

PCA(n_components=32)

In [37]:
pickle.dump(pca, open("models/pca.pkl","wb"))

In [14]:
class EmbedDataset(Dataset):
    def __init__(self, x, y, embeder,
                 decompositor=None,
                 **kwargs):
        assert len(x) == len(y)
        
        self.embeder = embeder
        self.x_opposite = x[y == 0.]
        self.x_target = x[y == 1.]
        self.y_opposite = y[y == 0.]
        self.y_target = y[y == 1.]
        self.decompositor = decompositor
        
        
    def __len__(self):
        return len(self.y_opposite) + len(self.y_target)
    
    
    def __getitem__(self, _):
        if random.random() > 0.5:
            target = False
            idx = random.choice(range(len(self.x_opposite)))
            xid, y = self.x_opposite[idx], self.y_opposite[idx]
            
        else:
            target = True
            idx = random.choice(range(len(self.x_target)))
            xid, y = self.x_target[idx], self.y_target[idx]
            
            
        x = self.embeder[int(xid)]
        if self.decompositor:
            x = self.decompositor(x).astype('float32')
        
        return x[0], y[None, ...]

In [15]:
class TestDataset(Dataset):
    def __init__(self, x, y, embeder,
                 decompositor=None,
                 **kwargs):
        assert len(x) == len(y)
        
        self.x = x
        self.y = y

        self.embeder = embeder
        self.decompositor = decompositor
        
        
    def __len__(self):
        return len(self.y)
    
    
    def __getitem__(self, idx):
        y = self.y[idx]
        x = self.embeder[int(self.x[idx])]
        
        if self.decompositor:
            x = self.decompositor(x).astype('float32')
            
        return x[0], y[None, ...]

In [16]:
class Model(nn.Module):
    def __init__(self, input_dim, d=[32, 32], **kwargs):
        super().__init__()
        
        seq = []
        d = [input_dim] + d + [1]
        for i in range(len(d)-1):
            seq.append(
                nn.Linear(d[i], d[i+1])
            )
            seq.append(nn.Dropout(p=0.5))
            if i != len(d)-2:
                seq.append(nn.GELU())
                
        self.seq = nn.Sequential(*seq)

        
    def forward(self, x: torch.Tensor):
        return self.seq(x)

In [17]:
class Trainer:
    def __init__(self, 
        model, loss_fn, optimizer, 
        stop_batch, metric=None,
        device='cuda', fp16=False, 
        **kwargs):
        self.model: nn.Module = model
        self.device = device
        self.metric = metric
        
        self.stop_batch = stop_batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer  
        
        self.fp16 = fp16
        if fp16:
            self.scaler = torch.cuda.amp.GradScaler()        
        
        
    def checkpoint(self) -> dict:
        cpoint =  {
            "model": self.model.state_dict(),
            "optimizer": self.optimizer.state_dict(),
        }
        if self.fp16:
            cpoint["scaler"] = self.scaler.state_dict()
            
        return cpoint
    
        
    def train(self, dataset, epoch) -> float:
        self.model.train()
        running_loss = 0
        for idx, batch in enumerate(dataset):
            X, Y = batch
            X, Y = X.to(self.device), Y.to(self.device)
            running_loss+= self.__train(X, Y)
            if idx >= self.stop_batch:
                break
                
        return running_loss
            
            
    def val(self, dataset) -> list[torch.Tensor]:
        self.model.eval()
        val_pred, val_true = [], []
        with torch.inference_mode():
            for batch in dataset:
                X, Y = batch
                val_pred+= [self.model(X.to(self.device)).cpu()]
                val_true+= [Y]
        return torch.cat(val_pred), torch.cat(val_true)
                
            
    def __train(self, X, Y) -> float:
        self.optimizer.zero_grad()
        if self.fp16:
            with torch.cuda.amp.autocast(enabled=True):
                outputs = self.model(X)
                loss = self.loss_fn(outputs, Y)
                
            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()

        else:
            outputs = self.model(X)
            loss = self.loss_fn(outputs, Y)    
            loss.backward()
            self.optimizer.step()
            
        if self.metric:
            self.metric(outputs.sigmoid(), Y.int())
            
        return loss.item()

In [18]:
trainLoader = torch.utils.data.DataLoader(
    EmbedDataset(
        x=Xtrain, 
        y=Ytrain, 
        embeder=embeder,
        decompositor=pca.transform), 
    batch_size=2048, 
)

In [19]:
valLoader = torch.utils.data.DataLoader(
    TestDataset(
        x=Xval, 
        y=Yval, 
        embeder=embeder,
        decompositor=pca.transform), 
    batch_size=2048, 
)

In [20]:
testLoader = torch.utils.data.DataLoader(
    TestDataset(
        x=Xtest, 
        y=Ytest, 
        embeder=embeder,
        decompositor=pca.transform), 
    batch_size=2048, 
)

In [21]:
model = Model(EMBED_DIM, d=[64, 64])
trainer = Trainer(
    model=model.cuda(),
    stop_batch=10_000/2048,
    metric=torchmetrics.AUROC(),
    loss_fn=nn.BCEWithLogitsLoss(reduce=True),
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4),
)

acc = torchmetrics.Accuracy()
auc = torchmetrics.AUROC()

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [22]:
torchsummary.summary(model);

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Linear: 2-1                       2,112
|    └─Dropout: 2-2                      --
|    └─GELU: 2-3                         --
|    └─Linear: 2-4                       2,080
|    └─Dropout: 2-5                      --
|    └─GELU: 2-6                         --
|    └─Linear: 2-7                       33
|    └─Dropout: 2-8                      --
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0


In [23]:
name = 'InceptionResnetV1 vggface2 pca 64'
board_name = name + datetime.datetime.now().strftime("%Y.%m.%d - %H-%M-%S")

log_dir = f"logs/fit/{board_name}"
writer = SummaryWriter(log_dir)

In [24]:
try:
    wait = 0
    patience = 50
    
    epoch = 0
    best_loss = -np.inf
    while wait < patience:
        train_loss = trainer.train(trainLoader, epoch)

        val_pred, val_true = trainer.val(valLoader)
        metrics = {
            'AUC': auc(val_pred.sigmoid(), val_true.int()),
            'ACC': acc(val_pred.sigmoid(), val_true.int()),
        }
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('AUC/train', trainer.metric.compute(), epoch)
        writer.add_scalar('AUC/val', metrics['AUC'], epoch)
        writer.add_scalar('ACC/val', metrics['ACC'], epoch)


        wait+=1
        epoch+=1
        if metrics['AUC'] > best_loss:
            checkpoint = trainer.checkpoint()
            torch.save(checkpoint, f'models/w/{name}.torch')
            best_loss = metrics['AUC']
            wait = 0


except KeyboardInterrupt:
    print("KeyboardInterrupt")

In [27]:
checkpoint = torch.load(f'models/w/{name}.torch')

In [29]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [45]:
test_pred, test_true = trainer.val(testLoader)

In [47]:
print('AUC:', auc(val_pred.sigmoid(), val_true.int()))
print('ACC:', acc(val_pred.sigmoid(), val_true.int()))

AUC: tensor(0.7045)
ACC: tensor(0.8362)
